<a href="https://colab.research.google.com/github/LennonJohnson/NFLPredictor/blob/main/NFLWinModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NFL Win Model Using Logistic Regression and EPA data


Run getData.py first to get epa data in an exponentially weighted rolling window

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:

#read training data
df = pd.read_csv("trainingData.csv")

#split data into training and testing
trainData = df.loc[df['season'] != 2023]
testData = df.loc[df['season'] == 2023]

trainFeatures = trainData.drop(columns=["home_team", "away_team", "season", "week", "home_score", "away_score", "home_margin"])
trainLabels = trainFeatures.pop("home_win")

testFeatures = testData.drop(columns=["home_team", "away_team", "season", "week", "home_score", "away_score", "home_margin"])
testLabels = testFeatures.pop("home_win")


In [ ]:
trainFeatures

,ewma_rushing_offense_epa_home,ewma_passing_offense_epa_home,ewma_rushing_defense_epa_home,ewma_passing_defense_epa_home,ewma_rushing_offense_epa_away,ewma_passing_offense_epa_away,ewma_rushing_defense_epa_away,ewma_passing_defense_epa_away
0,0.089867,0.107113,-0.061915,-0.012979,-0.087773,0.012560,-0.118844,0.161396
1,-0.045242,-0.156405,-0.115364,0.202096,-0.140480,-0.070151,-0.089373,0.009090
2,0.063742,0.172596,-0.066666,-0.182212,-0.038830,0.002799,0.060918,0.118202
3,-0.154126,0.140918,-0.108520,0.021535,0.015915,0.135219,-0.130682,-0.040705
4,-0.185825,-0.047691,-0.035208,-0.141715,-0.039489,0.116518,0.055469,-0.061941
...,...,...,...,...,...,...,...,...
833,0.008992,0.189114,-0.025761,-0.121283,-0.097486,0.046705,-0.136587,-0.012219
834,0.032653,0.001956,0.051249,-0.103904,0.163642,0.053687,0.066741,-0.003245
835,0.007926,0.215704,0.061466,-0.119131,-0.044719,0.140419,-0.099634,-0.027013
836,0.097051,0.044449,0.065952,-0.167419,-0.024541,0.281491,-0.085682,-0.106105


In [ ]:
#convert features into numpy arrays
trainFeatures = np.array(trainFeatures)
testFeatures = np.array(testFeatures)

trainLabels = np.array(trainLabels)
testLabels = np.array(testLabels)

Create Model

In [ ]:
clf = LogisticRegression()
clf.fit(trainFeatures, trainLabels)

LogisticRegression()

Evaluate Accuracy

In [ ]:

accuracy_scores = cross_val_score(clf, trainFeatures, trainLabels, cv=10)
log_losses = cross_val_score(clf, trainFeatures, trainLabels, cv=10, scoring='neg_log_loss')

print('Model Accuracy:', np.mean(accuracy_scores))

print('Neg log loss:', np.mean(log_losses))

Model Accuracy: 0.6194492254733219
Neg log loss: -0.6477011036987896


Predict Games using Test Data

In [ ]:

predictions = clf.predict(testFeatures)
probabilities = clf.predict_proba(testFeatures)

correct = 0
numOfGames = len(testLabels)

for i in range(numOfGames):
    if testLabels[i] == predictions[i]:
        correct+=1

accuracy = correct/numOfGames * 100

print("Accuracy: " + str(accuracy) + "%")

Accuracy: 60.35087719298245%
